In [67]:
import requests
import json
import time
import pandas as pd
from IPython.display import clear_output

loop_count = 250
beer_count = 0

beer_df = pd.DataFrame(columns=['id','name','abv','ph','ibu', 'first_brewed', 'hops'])

for x in range(150, loop_count+1):
    #Use IPython custome clear output to minimise outputs
    clear_output(wait=True)
    
    #Get JSON using URL Request
    url_api = 'https://api.punkapi.com/v2/beers/' + str(x)
    response = requests.get(url=url_api, headers = {'Content-Type': 'application/json', 'Accept': 'application/json'})
    
    #If error, API will return differently structured JSON rather than a HTTP error
    try:
        #Store JSON, grabbing first element (due to JSON structure from API)
        json_beer = json.loads(response.text)[0]
    except:
        #Error - so skip (but show ID with error)
        print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far! No beer with ID: ",x)
        continue
        
    #Assign beer data into Dictionary for later analysis
    beer_dict = {}
    
    #Create lists for hops and yeast
    hops_list = []
    yeast_list = []
    
    #Store beer info from JSON
    beer_dict['id'] = json_beer['id']
    beer_dict['name'] = json_beer['name']
    beer_dict['abv'] = json_beer['abv']
    beer_dict['ph'] = json_beer['ph']
    beer_dict['ibu'] = json_beer['ibu'] 
    beer_dict['first_brewed'] = json_beer['first_brewed']
    
    #Use a List comprehension to get all hops types per beer
    hops_list = [hop['name'] for hop in json_beer['ingredients']['hops']]
    
    #Remove hop duplicates (by creating a set) and put into array
    hops_list = list(set(hops_list))
    beer_dict['hops'] = hops_list
    
    #Append to dataframe
    temp_df  = pd.DataFrame([beer_dict], columns=beer_list.keys())
    beer_df = beer_df.append(temp_df)
    
    #Add beer count
    beer_count  += 1
    
    #Print progress
    print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far!")
    
    #Wait fixed time to prevent API throttling / restricting usage
    time.sleep(.75)

#Reset index of Dataframe (ensuring that old index doesn't get added as a column)
beer_df = beer_df.reset_index(drop=True)

Grabbing Punk beer data. Current progress: 250 / 250 .  101  beer(s) discovered so far!


In [78]:
import pandas as pd

#Create new dataframe to understand hops brewed by year
hops_df = pd.DataFrame(columns=['name', 'year'])

#Iterate each row in the raw beer dataframe
for index, row in beer_df.iterrows():
    
    #As multiple hops can be in one brew, second loop to iterate through list within beer dataframe row
    for hop_name in row['hops']:
        
        #Create temporary dictionary to be used to append to the hops dataframe
        temp_dict = {}

        #Get name of hop
        temp_dict['name'] = hop_name
        
        #Get year of brew, difficulty here is inconsistent formatting in data (sometimes MM/YYYY or YYYY)
        #Therefore search for "/" and split if required 
        temp_dict['year'] = beer_df.loc[index, 'first_brewed']
        if "/" in temp_dict['year']:
            temp_dict['year'] = temp_dict['year'].split("/")[1]
            
        #Append temperorary dictionary to hops dataframe
        temp_df  = pd.DataFrame([temp_dict], columns=hops_df.keys())
        hops_df = hops_df.append(temp_df)

#Reset index of Dataframe (ensuring that old index doesn't get added as a column)
hops_df = hops_df.reset_index(drop=True)  

#Change hops dataframe structure - years as columns and hop name as index
hops_df = pd.crosstab(index=hops_df['name'], columns=hops_df['year'])

#Print progress
print("Hops dataframe completed!")

Hops dataframe completed!


In [75]:
hops_df

year,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
name,,,,,,,,,,
1 lemon drop chilli,0,0,0,0,0,0,0,0,0,1
Ahtanum,1,0,0,0,0,0,0,1,0,0
Amarillo,1,0,1,0,1,3,2,6,6,9
Amarilo,0,0,0,0,0,0,0,0,0,1
Apollo,0,0,0,0,0,0,0,0,0,1
Bay Leaves,0,0,0,0,0,1,0,0,0,0
Blueberries,0,0,0,0,0,0,1,0,0,0
Bramling Cross,0,1,0,0,1,0,1,0,2,6
Bravo,0,0,0,0,0,0,2,0,0,0
